# 例：サンプルホテルおよびフライト予約エージェント

このソリューションは、フライトチケットとホテルの予約に役立ちます。シナリオは、2024年2月20日にロンドン・ヒースロー（LHR）からニューヨーク（JFK）へのエコノミークラスのブリティッシュ・エアウェイズ便での往復旅行で、帰りは2025年2月27日です。ニューヨークのヒルトンホテルに滞在したいので、フライトとホテルの料金を教えてください。


# Azure AI Agent Serviceを初期化し、**.env**から構成情報を取得する

### **.env**

.envファイルを作成します。

**.env**には、Azure AI Agent Serviceの接続文字列、AOAIで使用されるモデル、および対応するGoogle API検索サービスのAPIやENDPOINTなどが含まれます。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "あなたのAzure AI Agent Serviceモデルのデプロイメント名"

[**NOTE**] 100,000のレート制限（1分あたりのトークン数）、600のレート制限（1分あたりのリクエスト数）を持つモデルが必要です。

  Microsoft Foundry - Model and Endpointでモデルを取得できます。


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "あなたのAzure AI Agent Serviceプロジェクト接続文字列"

  AI Foundryポータル画面のプロジェクト概要でプロジェクト接続文字列を取得できます。

- **SERPAPI_SEARCH_API_KEY** = "あなたのSERPAPI検索APIキー"
- **SERPAPI_SEARCH_ENDPOINT** = "あなたのSERPAPI検索エンドポイント"

Azure AI Agent Serviceのモデルデプロイメント名とプロジェクト接続文字列を取得するには、Azure AI Agent Serviceを作成する必要があります。直接作成するには[このテンプレート](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)の使用を推奨します（***注意:*** Azure AI Agent Serviceは現在限定されたリージョンに設定されています。リージョン設定は[こちらのリンク](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)を参照してください）

AgentはSERPAPIにアクセスする必要があります。[このリンク](https://serpapi.com/searches)から登録することを推奨します。登録後、固有のAPIキーとENDPOINTを取得できます。


# セットアップ

このノートブックを実行するには、`pip install -r requirements.txt` を実行して必要なライブラリがインストールされていることを確認してください。


In [ ]:
from semantic_kernel import __version__

__version__

あなたのSemantic Kernelのバージョンは少なくとも1.27.2である必要があります。


.envファイルの設定とリソースを読み込んでください。キーと設定を追加し、ローカルの.envファイルを作成していることを確認してください。


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azureにログインする

次にAzureにログインする必要があります。ターミナルを開き、次のコマンドを実行してください：

```bash
az login
```

このコマンドはAzureの認証情報の入力を求め、Azure AIエージェントサービスが正しく機能するようにします。


# 説明:
これは、SERP（検索エンジン結果ページ）APIサービスにアクセスするためのAPIキーを格納する変数です。APIキーは、アカウントに関連付けられたリクエストを認証するために使用される一意の識別子です。

目的: この行の目的は、SERP APIサービスへのリクエストを認証するために使用できるように、APIキーを変数に格納することです。APIキーはサービスにアクセスして検索を実行するために必要です。
SERP APIキーの取得方法: SERP APIキーを取得するには、https://serpapi.com で以下の一般的な手順に従ってください（ご利用の特定のSERP APIサービスによって正確な手順は異なる場合があります）:

SERP APIサービスを選択する: SerpAPI、Googleカスタム検索JSON APIなど、いくつかのSERP APIサービスがあります。ニーズに最も適したものを選択してください。

アカウントにサインアップする: 選択したSERP APIサービスのウェブサイトにアクセスし、アカウントにサインアップしてください。基本情報の提供やメールアドレスの確認が必要な場合があります。

APIキーを作成する: サインアップ後、アカウントにログインし、APIセクションまたはダッシュボードに移動します。新しいAPIキーを作成または生成するオプションを探してください。
APIキーを.envファイルにコピーしてください。


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# 説明:
BASE_URL: これはSERP APIエンドポイントのベースURLを格納する変数です。変数名のBASE_URLは、このURLがAPIリクエストの出発点であることを示す慣例として使用されています。
'https://serpapi.com/search':

これはBASE_URL変数に割り当てられた実際のURL文字列です。SERP APIを使用して検索クエリを実行するためのエンドポイントを表しています。

# 目的:
この行の目的は、SERP APIのベースURLを保持する定数を定義することです。このURLは検索操作を実行するためのAPIリクエストの構築の出発点として使用されます。

# 使用法:
ベースURLを変数に定義することで、SERP APIにリクエストを行う必要があるたびにコード内で簡単に再利用できます。これによりコードの保守性が向上し、複数の場所でURLをハードコーディングすることによるエラーのリスクが減少します。現在の例は https://serpapi.com/search?engine=bing で、Bing検索APIを使用しています。異なるAPIは https://Serpapi.com で選択可能です。


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# 説明：

ここがあなたのプラグインコードがある場所です。

クラス定義：`class BookingPlugin`：ホテルとフライトの予約のためのメソッドを含むBookingPluginという名前のクラスを定義します。

ホテル予約メソッド：

- `@kernel_function(description="booking hotel")`：ホテル予約のカーネル関数として関数を説明するデコレーターです。
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`：注釈付きのパラメータと戻り値の型を持つホテル予約のためのメソッドを定義します。

このメソッドはホテル予約リクエストのパラメータ辞書を構築し、SERP APIにGETリクエストを送信します。レスポンスのステータスを確認し、成功した場合はホテルのプロパティを返し、リクエストが失敗した場合はNoneを返します。

フライト予約メソッド：

- `@kernel_function(description="booking flight")`：フライト予約のカーネル関数として関数を説明するデコレーターです。
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`：注釈付きのパラメータと戻り値の型を持つフライト予約のためのメソッドを定義します。

このメソッドは、往路と復路のフライトリクエストのパラメータ辞書を構築し、SERP APIにGETリクエストを送信します。レスポンスのステータスを確認し、成功した場合はフライト情報を結果文字列に追加し、リクエストが失敗した場合はエラーメッセージを表示します。メソッドはフライト情報を含む結果文字列を返します。


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# 説明:
インポート文: Azureの資格情報、AIエージェント、チャットメッセージの内容、作成者の役割、カーネル関数デコレーターに必要なモジュールをインポートします。

非同期コンテキストマネージャー: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): これはAzureの資格情報を処理し、AIエージェントクライアントを作成するための非同期コンテキストマネージャーを設定します。

エージェント名と指示: 
- `AGENT_NAME = "BookingAgent"`: エージェントの名前を定義します。
- `AGENT_INSTRUCTIONS = """..."""`: エージェントが予約リクエストを処理する方法についての詳細な指示を提供します。

エージェント定義の作成: `agent_definition = await client.agents.create_agent(...)`: 指定されたモデル、名前、および指示を使用してエージェント定義を作成します。

AzureAIエージェントの作成: `agent = AzureAIAgent(...)`: クライアント、エージェント定義、および定義されたプラグインを使ってAzureAIエージェントを作成します。

スレッドの作成: `thread: AzureAIAgentThread | None = None`: エージェントのスレッドを作成します。最初にスレッドを作成する必要はなく、`None`を指定すると、最初の呼び出し時に新しいスレッドが作成され、レスポンスの一部として返されます。

ユーザー入力: `user_inputs = ["..."]`: エージェントが処理するためのユーザー入力のリストを定義します。

finallyブロックでスレッドとエージェントを削除してリソースをクリーンアップします。


# 認証

`DefaultAzureCredential` クラスは Azure SDK for Python の一部です。Azure サービスに認証するためのデフォルトの方法を提供します。環境変数、マネージド ID、Azure CLI 資格情報など、特定の順序で複数の方法を使用して認証を試みます。

非同期操作: aio モジュールは DefaultAzureCredential クラスが非同期操作をサポートしていることを示します。これは、asyncio と共に使用してノンブロッキングの認証リクエストを実行できることを意味します。


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の確保に努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文はあくまで正式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳をお勧めします。本翻訳の利用に起因するいかなる誤解や解釈違いについても、当方は責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
